In [ ]:
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;
using System.IO;

In [ ]:
static async Task<List<string>> ListFilesAsync(HttpClient http, string owner,string repo, string path)
{
    var result = new List<string>();
    var apiUrl = $"https://api.github.com/repos/{owner}/{repo}/contents/{path}";
    http.DefaultRequestHeaders.UserAgent.ParseAdd("CSharpApps"); 

    var resp = await http.GetStringAsync(apiUrl);
    using var doc = JsonDocument.Parse(resp);

    foreach (var item in doc.RootElement.EnumerateArray())
    {
        var type = item.GetProperty("type").GetString();
        if (type == "file")
        {
            // raw URL:
            var rawUrl = item.GetProperty("download_url").GetString();
            result.Add(rawUrl);
        }
        else if (type == "dir")
        {
            // recurse into subdirectory
            var subPath = item.GetProperty("path").GetString();
            var subUrls = await ListFilesAsync(http, owner, repo, subPath);
            result.AddRange(subUrls);
        }
    }

    return result;
}

In [ ]:
// Change these to your target repo/owner/path:
const string Owner  = "abhijeetunreal";
const string Repo   = "3DB";
const string Path   = "model"; 
var http = new HttpClient();

var urls = await ListFilesAsync(http, Owner, Repo, Path);
foreach (var u in urls)
    Console.WriteLine(u);

In [ ]:
public class DataRequestModel
{
    public bool IsFavourite { get; set; } = false;
    public string Name { get; set; } = "CFD Models Name";
    public string Thumbnail { get; set; } = "https://codefrydev.in/images/IconCodefrydev.svg";
    public string Description { get; set; } = "A Nerve Plant";

    public string Source { get; set; } =
        "https://raw.githubusercontent.com/abhijeetunreal/3DB/refs/heads/main/model/nerveplant.glb";

    public string Poster { get; set; } = "poster.webp";
    public string AltText { get; set; } = "3D Model";
    public string IosSrc { get; set; } = "";

    public List<string> AnimationNames { get; set; } =[];

    public List<string> EnvironmentImages { get; set; } =["https://modelviewer.dev/shared-assets/environments/spruit_sunrise_1k_HDR.jpg"];

    public List<string> SkyboxImages { get; set; } =["https://modelviewer.dev/shared-assets/environments/spruit_sunrise_1k_HDR.jpg"];
}

In [ ]:
var listOfContent = new Dictionary<string,DataRequestModel>();
foreach (var u in urls)
{
    var fileName = u.Split('/').Last();
    var folderName = u.Split('/')[u.Split('/').Length - 2]; 
    listOfContent.TryAdd(folderName, new DataRequestModel());
    listOfContent[folderName].Name = folderName;

    if(fileName.Contains("poster"))
    {
        listOfContent[folderName].Poster = fileName;
    }
    else if (fileName.Contains("ios"))
    {
        listOfContent[folderName].IosSrc = fileName;
    }
    else if (fileName.Contains("alt"))
    {
        listOfContent[folderName].AltText = fileName;
    }
    else if (fileName.Contains("favourite"))
    {
        listOfContent[folderName].IsFavourite = true;
    }
    else if (fileName.Contains("description"))
    {
        listOfContent[folderName].Description = fileName;
    }
    else if(fileName.Contains(".glb"))
    {
        listOfContent[folderName].Source = u;
    }
    else if (fileName.Contains("Environment"))
    {
        listOfContent[folderName].EnvironmentImages.Add(u);
    }
    else if (fileName.Contains("Thumbnail"))
    {
        listOfContent[folderName].Thumbnail = u;
    }
    else if (fileName.Contains("Skybox"))
    {
        listOfContent[folderName].SkyboxImages.Add(u);
    }
    else if (fileName.Contains("Animation"))
    {
        var animationsName = await new HttpClient().GetStringAsync(u);
        var animationName = animationsName.Split('\n').Select(x=>x.Trim()).Where(x=>x.Length>0).ToList();
        listOfContent[folderName].AnimationNames= animationName;
    }
}
listOfContent.Values.ToList()

In [ ]:
using System.Text.Json.Serialization;
var text = JsonSerializer.Serialize(listOfContent.Values.ToList(), new JsonSerializerOptions
{
    WriteIndented = true
});
text

In [ ]:
File.WriteAllText("index.json", text);

In [ ]:
Experimental

In [ ]:
using System.Text.Json;
using ThreeDAssets.Components;
using ThreeDAssets.Models;
using Constant = ThreeDAssets.Utility.Constant;

namespace ThreeDAssets.Services;

public class AssetsService
{
    private readonly HttpClient _httpClient;

    public AssetsService(HttpClient httpClient)
    {
        _httpClient = httpClient;
        _httpClient.DefaultRequestHeaders.UserAgent.ParseAdd("CSharpApp");
    }

    private async Task<List<string>> ListFilesAsync(string owner, string repo, string path)
    {
        var result = new List<string>();
        var apiUrl = $"https://api.github.com/repos/{owner}/{repo}/contents/{path}";

        var response = await _httpClient.GetStringAsync(apiUrl);
        using var doc = JsonDocument.Parse(response);

        foreach (var item in doc.RootElement.EnumerateArray())
        {
            var type = item.GetProperty("type").GetString();
            var downloadUrl = item.GetProperty("download_url").GetString();
            var subPath = item.GetProperty("path").GetString();

            if (type == "file")
            {
                if (!string.IsNullOrEmpty(downloadUrl))
                    result.Add(downloadUrl);
            }
            else if (type == "dir")
            {
                // recurse
                var nested = await ListFilesAsync(owner, repo, subPath!);
                result.AddRange(nested);
            }
        }

        return result;
    }

    public async Task<List<DataRequestModel>> GetAssetsAsync()
    {
        // Ensure Constant.Owner is spelled correctly in your constants
        var urls = await ListFilesAsync(Constant.Owner, Constant.Repo, Constant.Path);

        var assets = new Dictionary<string, DataRequestModel>();

        foreach (var url in urls)
        {
            var segments = url.Split('/');
            var fileName = segments.Last();
            var folder = segments[^2];

            if (!assets.ContainsKey(folder))
            {
                assets[folder] = new DataRequestModel
                {
                    Name = folder,
                    EnvironmentImages = new List<string>(),
                    SkyboxImages = new List<string>()
                };
            }

            var model = assets[folder];

            // Map by suffix or type
            if (fileName.Contains("poster", StringComparison.OrdinalIgnoreCase))
                model.PosterUrl = url;
            else if (fileName.Contains("ios", StringComparison.OrdinalIgnoreCase))
                model.IosSrcUrl = url;
            else if (fileName.Contains("alt", StringComparison.OrdinalIgnoreCase))
                model.AltText = await _httpClient.GetStringAsync(url);
            else if (fileName.Contains("favourite", StringComparison.OrdinalIgnoreCase))
                model.IsFavourite = true;
            else if (fileName.EndsWith(".glb", StringComparison.OrdinalIgnoreCase))
                model.SourceUrl = url;
            else if (fileName.Contains("Thumbnail", StringComparison.OrdinalIgnoreCase))
                model.ThumbnailUrl = url;
            else if (fileName.Contains("Environment", StringComparison.OrdinalIgnoreCase))
                model.EnvironmentImages.Add(url);
            else if (fileName.Contains("Skybox", StringComparison.OrdinalIgnoreCase))
                model.SkyboxImages.Add(url);
            else if (fileName.Contains("description", StringComparison.OrdinalIgnoreCase))
                model.Description = await _httpClient.GetStringAsync(url);
            else if (fileName.Contains("Animation", StringComparison.OrdinalIgnoreCase))
            {
                var content = await _httpClient.GetStringAsync(url);
                model.AnimationNames = content
                    .Split('\n', StringSplitOptions.RemoveEmptyEntries)
                    .Select(s => s.Trim())
                    .ToList();
            }
        }

        return assets.Values.ToList();
    }
}
